# PROBLEM 1

## Part a

Basically, we choose important features to recommend an item to users based on their purchase, a kind of content-based system. In my opinion, all features that help provide more information are important. In the current trend, researchers usually extend the size and the architecture to help models can deeply understand the connection between the features. But for this case, I will derive the important features below:
- from user: age, sex, nationality
- Product type
- Title + Description: Can help to recommend items with similar or better functions (retrieved from text), sometimes it doesn't have the same product type.
- Image: to recommend similar or better-looking items.
- purchased timestamp may help.

## Part b

In this case, we have the information of items that the user is interested in. So we gonna recommend new items based on items similarity using: 
- Item title + description: Tfidf, BM25, deep learning context meaning (BERT, RoBERTa, ...)
- Image: CNN architecture

<br>Apply reranking to get better embedding.

# PROBLEM 2

## Part a

The work of Achlioptas only focuses on handling the cases of s = 1 and s = 3. With s = 3, only $1/3$ of the data needs to be processed and the memory can be reduced. From that, Very Sparse Random Projections extend the problem and make it more general. They tried to prove that one can use other larger factors like $\sqrt{D}$ and still keep the performance. This means it only needs to process $\frac{1}{\sqrt{D}}$ of the data with little loss in accuracy

Pros and cons of using big values of parameter s in very sparse random projection methods:
- Pros: can achieve an s-fold speedup, save a lot of memory.
- Cons: loss of the information of features when having too many zero-elements in projection matrix.

## Part b

Because the requirement is quite matched with the idea of 'very sparse random projections'. We have a large dimension of features, we want to project it to a lower dimension. And I think just a small subset of sites was viewed by user, so maybe the features vector contains a lot of zero, so basically, we have a sparse matrix as the data. It can help to speed up the multiplication between 2 sparse matrixes using some libs, and it also reduces concerns about losing information when used with larger factors.

The parameters I want to use: |U|x|S|, d, A(u,s), factor

# PROBLEM 3

## part a

The fundamental differences between mean and median:
- mean was used to describe a <b>central tendency</b> by adding all list's values and then dividing by the length of list. It can say what value point is closest with <b>all</b> elements of the list.
- median was used to find a <b>central value</b> in a sorted list.

About housing prices in District 4, Ho Chi Minh City. I checked the prices in batdongsan.com and by my real estate knowledge, I'm quite sure that the mean would be higher than the median. 
<br>Because mean will be affected by extreme observations like too great values or too small values. And in real estate, we will see some so high prices like a penthouse or large plot of land, ...

## part b

We can use quickselect. The algorithm was used to find k-th smallest/largest element with O(n) in average case but O(n^2) in worst case. Can use the algo from https://en.wikipedia.org/wiki/Median_of_medians to get a better O(n) in worst case. But in practice, a lot of documents describe that it's not good on average.

# PROBLEM 4

## part a

It seem they used sort-merge join for the left join. So the computational complexity of sort is O(n log n), index lookup is O(1), so O(n) for n elements. So in this case, we got O(n log n).

## part b

Hash join seems suitable for this case. The computational complexity gonna be O(n).

In [44]:
import pandas as pd
from numpy import nan as Nan

df1 = pd.DataFrame({'usr_id': ['K0', 'K1', 'K2', 'K3', 'K4', 'K5'],
                   'feature_1': [3, 5, 1, 6, 6, 8],
                   'feature_2': [1, 9, 2, 6, 4, 10],
                   'feature_3': [0, 9, 13, 5, 2, 12]})
df2 = pd.DataFrame({'usr_id': ['K2', 'K1', 'K0', 'K3', 'K4', 'K6'],
                     'feature_4': [123, 55, 1, 11, 4, 1],
                     'feature_5': [1, 12, 4, 6, 89, 100]})

In [47]:
def hash_left_join(df1, df2, on='usr_id', lsuffix='l_', rsuffix='r_'):
    
    def get_key_dict(df, on='usr_id'):
        return {value: index for index, value in enumerate(df[on].values)}
    
    df2_key_dict = get_key_dict(df2, on='usr_id') # O(n)
    # process keys
    df1_keys = df1.keys()
    df2_keys = df2.keys()
    new_keys = [on]
    for key in df1_keys:
        if key == on:
            continue
        new_key = lsuffix + key if key != on and key in df2_keys else key
        new_keys.append(new_key)
        
    for key in df2_keys:
        if key == on:
            continue
        new_key = rsuffix + key if key != on and key in df1_keys else key
        new_keys.append(new_key)
    
    # contruct new join df
    rows_value = []
    # loop every row in df1 => O(n)
    for idx, value in enumerate(df1[on].values):
        row_value = [value]
        row_value.extend([df1[key][idx] for key in df1_keys if key != on])
        if value in df2_key_dict: # O(1)
            row_value.extend([df2[key][idx] for key in df2_keys if key != on])
        else:
            row_value.extend([Nan for key in df2_keys if key != on])
        rows_value.append(row_value)
    return pd.DataFrame(rows_value, columns=new_keys)

In [48]:
hash_left_join(df1, df2)

,usr_id,feature_1,feature_2,feature_3,feature_4,feature_5
0,K0,3,1,0,123.0,1.0
1,K1,5,9,9,55.0,12.0
2,K2,1,2,13,1.0,4.0
3,K3,6,6,5,11.0,6.0
4,K4,6,4,2,4.0,89.0
5,K5,8,10,12,NaN,NaN


# PROBLEM 5

## Part a: Scenario [S01]: n < 1,000

Because n is quite small so I think just my first idea is enough. Firstly, sort the all values by using app column.Then, use groupby to group the row by using usr_id column. We can handle first 2 steps with 2 lines of code. After that, make a new dataframe and transform to the target output.
<br> sort: O(n log n)
<br> groupby: O(n)
<br> => O(n log n)

## Part b: Scenario [S02]: n > 10,000

With n very large, let reduce the sort step. The role of the sort step is mapping frequencies with new column names. So we can replace it by using dict to map the app value to column names. 
<br>Build the dict: O(m) with m is the length of df01
<br>Group by usr_id: O(m)
<br>Map the value to new column: O(1) for each get from dict => O(n) for n apps
<br>=> O(m + n)